<a href="https://colab.research.google.com/github/ChoeTaeBin/deep_learning_study/blob/master/autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

Tensor의 requires_grad 속성을 True로 설정 하면, autograd기능을 이용할 수 있다.



In [3]:
x = torch.ones(2, 3, requires_grad = True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


requires_grad == True 성질은 유전 된다. 즉, x를 통해 만든 y도 requires_grad == True가 된다.  
또 자신을 만든 연산의 역연산을 grad_fn에 기록해둔다. 이는 back propagation에 이용하기 위함이다.  

In [7]:
y = x + 2
print(y)
print(f"y.requires_grad:{ y.requires_grad}")

tensor([[3., 3., 3.],
        [3., 3., 3.]], grad_fn=<AddBackward0>)
y.requires_grad:True


In [11]:
z = y.mean()
print(z)
print(f"z.requires_grad:{ z.requires_grad}")


tensor(3., grad_fn=<MeanBackward0>)
z.requires_grad:True


z.backward() 는 z로 부터 back propagation을 한다는 이야기이다. 이때 z는 스칼라여야 한다.  

이후 x.grad에는 z를 x로 미분한 값이 저장된다.   
맨 처음 시작점이 아니므로 y.grad는 저장되지 않는다.

In [15]:
z.backward()

In [16]:
x.grad

tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]])

조금더 복잡한 연산을 정의하여 사용 해 보고 back propagation이 잘 작동하는지 확인 해 보자

In [20]:
def calc(a):
  b = a*2 + 1
  c = b*b
  d = c/(c+2)
  e = d.mean()
  return e

In [23]:
x = torch.tensor([1., 2., 3.], requires_grad = True)
y = calc(x)
y.backward()
print(x.grad)

tensor([0.0661, 0.0183, 0.0072])


검증

In [33]:
delta = 0.001

y = calc(torch.tensor([1.0, 2.0, 3.0]))

y_1 = calc(torch.tensor([1.0 + delta, 2.0, 3.0]))
y_2 = calc(torch.tensor([1.0, 2.0 + delta, 3.0]))
y_3 = calc(torch.tensor([1.0, 2.0, 3.0 + delta]))

torch.stack(((y_1-y)/delta, (y_2-y)/delta, (y_3-y)/delta))

tensor([0.0660, 0.0183, 0.0072])

거의 같은 값을 보이므로 미분값을 잘 계산한다고 볼 수 있다.